In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import auraloss
import IPython.display as ipd
import matplotlib.pyplot as plt
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F

from neural_field_synth.data import NSynthDataset
from neural_field_synth.signal import FIRNoiseSynth
from neural_field_synth.models import NeuralFieldSynth, LightningWrapper

<IPython.core.display.Javascript object>

In [3]:
model = NeuralFieldSynth(
    instrument_embedding_size=8,
    mlp_hidden_size=1024,
    mlp_hidden_layers=5,
    field_hidden_size=384,
    field_hidden_layers=3,
    wave_field_first_omega_0=300,
    wave_field_hidden_omega_0=100,
    noise_field_first_omega_0=100,
    noise_field_hidden_omega_0=30,
    noise_ir_length=128,
    noise_window_length=128,
    noise_hop_length=64,
    #     freeze_siren=True,
)
model = LightningWrapper(
    model,
    auraloss.freq.MultiResolutionSTFTLoss(
        fft_sizes=[64, 128, 256, 512, 1024, 4096],
        hop_sizes=[32, 64, 128, 256, 512, 2048],
        win_lengths=[64, 128, 256, 512, 1024, 4096],
    ),
    learning_rate=1e-3,
    log_audio=False,
)

<IPython.core.display.Javascript object>

In [4]:
ds = NSynthDataset("/import/c4dm-datasets/nsynth/nsynth-test/")
dl = torch.utils.data.DataLoader(
    ds, batch_size=4, num_workers=16, persistent_workers=True, shuffle=True
)

<IPython.core.display.Javascript object>

In [ ]:
trainer = pl.Trainer(
    #     overfit_batches=1,
    gpus=1,
    #     strategy=pl.plugins.DDPSpawnPlugin(find_unused_parameters=False),
)
trainer.fit(model, dl)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]

  | Name    | Type                    | Params
----------------------------------------------------
0 | model   | NeuralFieldSynth        | 17.7 M
1 | loss_fn | MultiResolutionSTFTLoss | 0     
----------------------------------------------------
17.7 M    Trainable params
0         Non-trainable params
17.7 M    Total params
70.957    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

In [ ]:
dl = torch.utils.data.DataLoader(
    ds, batch_size=8, num_workers=16, persistent_workers=True, shuffle=True
)
it = iter(dl)

In [ ]:
batch = next(it)

net = model.model.cuda(0).eval()

target = batch["audio"].float().cuda(0)
instrument = batch["instrument"].float().cuda(0)
pitch = batch["pitch"].float().cuda(0)
velocity = batch["velocity"].float().cuda(0)

time = torch.linspace(-1, 1, target.shape[-1], device=target.device)[
..., None
].expand(-1, target.shape[0])

recon = net(time, pitch, velocity, instrument, return_params=True)


In [ ]:
for i in range(target.shape[0]):
    print("item %d" % i)
    r = recon.output[:, i].detach().cpu()

    print(" -- target")
    ipd.display(ipd.Audio(target[i, 0].cpu(), rate=model.model.sample_rate))
    print(" -- recon")
    ipd.display(ipd.Audio(r - r.mean(), rate=model.model.sample_rate))
    print(" ---- wavetable")
    ipd.display(
        ipd.Audio(
            recon.wavetable_signal.cpu().detach()[:, i, 0], rate=model.model.sample_rate
        )
    )
    print(" ---- noise")
    ipd.display(
        ipd.Audio(recon.noise_signal.cpu().detach()[:, i], rate=model.model.sample_rate)
    )

In [ ]:
recon.noise_signal.max()

In [ ]:
batch = next(it)

net = model.model.cuda(0)

target = batch["audio"].float().cuda(0)
instrument = batch["instrument"].float().cuda(0)
pitch = batch["pitch"].float().cuda(0)
velocity = batch["velocity"].float().cuda(0)

time = torch.linspace(-1, 1, target.shape[-1], device=target.device)[..., None].expand(
    -1, target.shape[0]
)

recon = net(time, pitch, velocity, instrument, return_params=True)

In [ ]:
ipd.Audio(recon.noise_signal[:, 0].detach().cpu(), rate=net.sample_rate)

In [ ]:
recon.noise_film_params[0][0, 0][0]

In [ ]:
model.model.cuda(0).noise_field(recon.fir_sample_signal, *recon.noise_film_params)[0:2]

In [ ]:
plt.stem(recon.impulse_response[100, 0].detach().cpu())